# Bayesian esimation of parameters with MCMC
# and prediction of values in test set

#### Initialisation

In [1]:
import numpy as np
import random

from pandas import read_csv
from multiprocess import Pool
from itertools import product

from func_util import create_path, sort_second, showl
from func_bayes import ModelBay, SEMmx
# from func_bfull_tmp import ModelBay, SEMmx




var_ordinal = ['FloCol', 'StemCol', 'FlowStemCol', 'SeedCol', 'PodShape', 'SeedShape',
               'BushShape', 'StemBr', 'StemBr1ord', 'StemBr2ord', 'StemL1ord', 'LeafSize',
              'PodD', 'PodS', 'AscoRes', 'Germ']


n_cv = 20  # Number of cross-validation sets
thresh = 5 # Number of latent factors

kappa_flag = True  # True - if you are using extended model; False - if base model
beta_flag = False  # True - if connected model; False - if zero model

path_cv_data = '../data/cv_data/'  # path with training and test sets
path_cv_model = '../data/cv_model/'  # path with modes and para,eters
path_mod_specif = path_cv_model + ('mod_grow/' if beta_flag else 'models/')  # path with model specification
path_priors = path_cv_model + 'priors_' + \
                ('full_' if beta_flag else 'zero_') + 'cut' + \
                ('_phen/' if kappa_flag else '/')  # prior values of parameters


path_mcmc = path_cv_model + 'mcmc_final_' + \
                ('full_' if beta_flag else 'zero_') + 'cut' + \
                ('_phen_' if kappa_flag else '_') # mcmc chains


# path_mcmc = path_cv_model + 'mcmc_' + \
#                 ('full_' if beta_flag else 'zero_') + 'cut' + \
#                 ('_phen_v' if kappa_flag else '_v') # mcmc chains

rseeds = [125, 137, 159, 257, 31]


## Run MCMC

In [3]:
def mcmc_individual(idata, nseed):
    print(idata, nseed)

    file_mod = ('mod_spart_' if beta_flag else 'mod_') + str(thresh) + '_' + str(idata) + '.txt'
    file_data = 'data_train_' + str(idata) + '.txt'
    file_prior = 'prior_' + str(thresh) + '_' + str(idata) + '.txt'
    file_mcmc =  'mcmc_' + str(thresh) + '_' + str(idata) + '.txt'

    # -------------------------------------------------------------------

    with open(path_mod_specif + file_mod, 'r') as f:
        mod = f.read()

    param_prior = read_csv(path_priors + file_prior, sep='\t', header=None)
    param_prior = param_prior.iloc[:, :3]
    data = read_csv(path_cv_data + file_data, sep='\t', header=0, index_col=0)
    # -------------------------------------------------------------------

    mod_bay = ModelBay(mod, data, param_prior, var_ordinal, kappa_flag=kappa_flag)
    self = mod_bay

    
    random.seed(nseed)
    self.optimise()


    # -------------------------------------------------------------------
    mcmc = mod_bay.mcmc
    np.savetxt(path_mcmc_seed + file_mcmc, mcmc)
    
def mcmc(idata):    
    for iseed in range(len(rseeds)):
        path_mcmc_seed = path_mcmc + '{}/'.format(iseed)
        create_path(path_mcmc_seed)
        
        mcmc_individual(idata, rseeds[iseed])
        
        
# for idata, iseed in product(range(n_cv), range(len(rseeds))):
    

n_thr = 5
with Pool(n_thr) as workers:
    pmap = workers.map
    pmap(mcmc, range(n_cv))



    

Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
1 125
0 125
3 125
4 125
2 125
10
10
10
10
10
(2, 268)
(2, 264)
(2, 263)
(2, 267)
(2, 279)
(3, 263)
(3, 268)
(3, 264)
(3, 267)
(3, 279)
(4, 263)
(4, 264)
(4, 268)
(4, 267)
(4, 279)
(5, 263)
(5, 268)
(5, 264)
(5, 267)
(5, 279)
(6, 263)
(6, 268)
(6, 264)
(6, 267)
(6, 279)
(7, 263)
(7, 264)
(7, 268)
(7, 267)


Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Traceback (most recent call last):
  File "/Users/anna/.conda/envs/cicer/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/anna/.conda/envs/cicer/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/anna/.conda/envs/cicer/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/anna/.conda/envs/cicer/lib/python3.7/site-packages/multiprocess/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/Users/anna/.conda/envs/cicer/lib/python3.7/site-packages/multiprocess/pool.py", line 44, in mapstar
    return list(map(*args))
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/anna/.conda/envs/cicer/lib/python3.7/s

KeyboardInterrupt: 

## Prediction

In [5]:
iseed = 0
path_mcmc_seed = path_mcmc + '{}/'.format(iseed)

# path_mcmc_seed = '../data/cv_model/mcmc_zero_cut_phen/'

data_res = []
data_test = []
for idata in range(n_cv):
    print(idata)


    file_mod = ('mod_spart_' if beta_flag else 'mod_') + str(thresh) + '_' + str(idata) + '.txt'
    file_data = 'data_train_' + str(idata) + '.txt'
    file_prior = 'prior_' + str(thresh) + '_' + str(idata) + '.txt'
    file_mcmc =  'mcmc_' + str(thresh) + '_' + str(idata) + '.txt'



    # -------------------------------------------------------------------

    with open(path_mod_specif + file_mod, 'r') as f:
        mod = f.read()

    param_prior = read_csv(path_priors + file_prior, sep='\t', header=None)
    param_prior = param_prior.iloc[:, :3]
    data = read_csv(path_cv_data + file_data, sep='\t', header=0, index_col=0)
    # -------------------------------------------------------------------

    mod_bay = ModelBay(mod, data, param_prior, var_ordinal, kappa_flag=kappa_flag)
    self = mod_bay

    # -------------------------------------------------------------------

    mcmc = read_csv(path_mcmc_seed + file_mcmc, sep=' ', header=None)
    x = np.mean(mcmc[100:1000], axis=0)
    self.param_val = x
    self.fill_mx()

    t_data = data.copy()
    tmp = self.predict_new(t_data)


    file_t_data = 'data_test_' + str(idata) + '.txt'
    t_data = read_csv(path_cv_data + file_t_data, sep='\t', header=0, index_col=0)
    y1, y2 = self.predict_new(t_data)

    data_res += [y1]
    data_test += [y2]

#     np.savetxt(path_predict + 'data_pred_' + str(idata) + '.txt', np.concatenate((y1,y2 ), axis=1), delimiter='\t')


0
BegFEndF 0.5646982467561252
EndFBegM 0.5155225108775182
Height 0.6966267618844973
Hlp 0.48214961778212767
NoPodsWeight 0.7427763566642829
PodLength 0.7417999198139446
PodWidth 0.735873566730491
PodsNumber 0.4254149104922327
PodsWeight 0.528862399291039
Seed1000W 0.7954449910171264
SeedsNumber 0.4503602003025861
SeedsWeight 0.49614060020346135
FloCol 0.7835422646496987
FlowStemCol 0.7380710324221236
SeedCol 0.7463734951785813
StemCol 0.7163889463815638


BegFEndF 0.49768055944633266
EndFBegM 0.09770489417728509
Height 0.3822670317020716
Hlp 0.3604799640177083
NoPodsWeight 0.5553350706087735
PodLength 0.6372770642174248
PodWidth 0.4790797025555695
PodsNumber 0.2306052251847871
PodsWeight 0.44533749725963223
Seed1000W 0.7328257232347448
SeedsNumber 0.38525161397643626
SeedsWeight 0.09261994582575962
FloCol 0.7073636609045911
FlowStemCol 0.8169023398448622
SeedCol 0.5716272003508365
StemCol 0.7541457760621392


1
BegFEndF 0.5605261510812435
EndFBegM 0.5248136065550453
Height 0.6889170696

BegFEndF 0.5726645170535367
EndFBegM 0.49614619326733517
Height 0.7175312527869641
Hlp 0.49426952590472084
NoPodsWeight 0.725631575921032
PodLength 0.7518314663731779
PodWidth 0.7202789650777439
PodsNumber 0.4072816274633815
PodsWeight 0.5334325430064019
Seed1000W 0.8002867402957656
SeedsNumber 0.4063906006209234
SeedsWeight 0.46586673325645767
FloCol 0.7987680789240964
FlowStemCol 0.7336656933373181
SeedCol 0.7739712409592245
StemCol 0.7180571265182314


BegFEndF 0.2035704921255116
EndFBegM 0.29238279199724576
Height 0.4959861759034523
Hlp 0.5489420907639173
NoPodsWeight 0.3328944656948085
PodLength 0.5289543955664275
PodWidth 0.5017407046452251
PodsNumber 0.25253984338008856
PodsWeight 0.25060649088281794
Seed1000W 0.5816027128116515
SeedsNumber 0.45169820246666187
SeedsWeight 0.286808125361487
FloCol 0.5588084653238379
FlowStemCol 0.6836942879973628
SeedCol 0.5934927890496908
StemCol 0.4502099035796325


10
BegFEndF 0.5281380159229941
EndFBegM 0.48322406769715787
Height 0.6910185293

BegFEndF 0.5660784866254407
EndFBegM 0.5269556735991724
Height 0.697182386701958
Hlp 0.4869951990191621
NoPodsWeight 0.7469735633144032
PodLength 0.7511229793812265
PodWidth 0.7146723895486683
PodsNumber 0.43883274198122296
PodsWeight 0.5329173754102914
Seed1000W 0.7853405414473222
SeedsNumber 0.4626723784108853
SeedsWeight 0.47571574156438895
FloCol 0.7857371215645861
FlowStemCol 0.7285321098151986
SeedCol 0.7444441869816814
StemCol 0.7249970204577476


BegFEndF 0.27477627647712477
EndFBegM 0.2254379111754907
Height 0.6974602188048409
Hlp 0.05106540997281134
NoPodsWeight 0.7205984889346312
PodLength 0.7059255315888059
PodWidth 0.7285735688791423
PodsNumber 0.16274101565729307
PodsWeight 0.4788555822443114
Seed1000W 0.847416847281636
SeedsNumber 0.16007440302915613
SeedsWeight 0.21130793924752617
FloCol 0.7542931025055625
FlowStemCol 0.7577454588534883
SeedCol 0.7877579789987728
StemCol 0.6543039642199359


19


FileNotFoundError: File b'../data/cv_model/mcmc_final_zero_cut_phen_0/mcmc_5_19.txt' does not exist

In [6]:
a1, a2 = tmp
print(np.round(y1[1,:]))
print(np.round(y2[1,:]))
# a2

[ 3.  2. 52. 24.  3. 23. 11.  4.  3.  6.  4.  3.  1.  4.  3.  4.]
[ 3.  2. 55. 17.  4. 23. 12.  4.  3.  6.  4.  3.  1.  3.  1.  3.]


In [9]:
all_pred = np.concatenate(data_res, axis=0)
all_test = np.concatenate(data_test, axis=0)
all_corr = np.corrcoef(np.transpose(all_pred), np.transpose(all_test))
showl(np.diag(all_corr,16))

0.27533912071877154
0.31971041497513275
0.5946308633101169
0.3279799702617948
0.6661730892748686
0.6787839770434279
0.6596242599063881
0.27128476714715144
0.43015299125158246
0.7426894352301407
0.3191270578496728
0.34666598166670387
0.7109632355929785
0.6757293545510734
0.675617367298963
0.671939743345269


In [ ]:
array([0.24888053, 0.28145095, 0.54057679, 0.33486997, 0.63808693,
       0.61764113, 0.58784906, 0.2657911 , 0.41796819, 0.65763619,
       0.33405266, 0.34761152, 0.66395458, 0.62377845, 0.65284014,
       0.64526531])

array([0.28310391, 0.32956473, 0.59651161, 0.34636718, 0.68216029,
       0.67513035, 0.65965216, 0.28962707, 0.45067788, 0.74368879,
       0.34205365, 0.36572123, 0.71528841, 0.67828805, 0.6791604 ,
       0.67297059])

array([0.25747447, 0.29276413, 0.54289802, 0.30213078, 0.64334839,
       0.62685117, 0.59260723, 0.26679164, 0.40977836, 0.66723869,
       0.32517067, 0.35461509, 0.66298821, 0.624358  , 0.65772425,
       0.64180852])
array([0.25585993, 0.28675712, 0.53486914, 0.32367197, 0.64375936,
       0.62684117, 0.59218922, 0.26780099, 0.40954181, 0.66786844,
       0.32533397, 0.3558123 , 0.66250535, 0.62218558, 0.65663463,
       0.64128996])

In [21]:
np.sqrt(np.mean((all_pred - all_test) ** 2, axis=0)) / np.mean(all_test, axis=0)
# showl(list(zip(all_pred[:,15],all_test[:,15])))

array([0.10061381, 0.20811557, 0.13454999, 0.19349928, 0.23298766,
       0.10063718, 0.13420941, 0.14097103, 0.26021066, 0.04688802,
       0.1734247 , 0.34845217, 0.31715207, 0.31097929, 0.52816507,
       0.29679181])

In [32]:
mod_bay.v_phen

['BegFEndF',
 'EndFBegM',
 'Height',
 'Hlp',
 'NoPodsWeight',
 'PodLength',
 'PodWidth',
 'PodsNumber',
 'PodsWeight',
 'Seed1000W',
 'SeedsNumber',
 'SeedsWeight',
 'FloCol',
 'FlowStemCol',
 'SeedCol',
 'StemCol']

In [20]:
all_test[:,1]

array([2.07944154, 1.60943791, 2.39789527, 2.99573227, 2.19722458,
       2.77258872, 2.83321334, 2.56494936, 2.89037176, 2.19722458,
       2.56494936, 2.77258872, 2.56494936, 1.79175947, 2.48490665,
       1.94591015, 2.39789527, 1.94591015, 1.94591015, 2.19722458,
       2.07944154, 2.30258509, 2.48490665, 2.63905733, 1.94591015,
       3.25809654, 1.60943791, 2.39789527, 2.19722458, 1.79175947,
       2.7080502 , 1.79175947, 1.38629436, 1.60943791, 2.19722458,
       2.39789527, 2.48490665, 2.48490665, 1.60943791, 2.30258509,
       2.48490665, 1.94591015, 2.30258509, 1.94591015, 2.56494936,
       2.63905733, 2.77258872, 2.7080502 , 1.79175947, 1.38629436,
       2.39789527, 2.39789527, 2.07944154, 3.09104245, 1.94591015,
       2.19722458, 1.79175947, 1.38629436, 1.60943791, 2.39789527,
       2.48490665, 3.21887582, 1.79175947, 2.39789527, 2.48490665,
       1.94591015, 2.63905733, 2.89037176, 2.83321334, 2.89037176,
       1.79175947, 1.60943791, 2.77258872, 2.30258509, 2.07944